In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import t as tstats
from scipy.stats.mstats import winsorize
from scipy.stats import pearsonr
from scipy.stats import wilcoxon
import warnings
warnings.filterwarnings("ignore")

/databricks/python/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# dbutils.fs.rm("dbfs:/path/tenure_au_eid_0610",recurse=True)
# dbutils.fs.rm("dbfs:/path/tenure_citepct_0709.csv",recurse=True)
# dbutils.fs.rm("dbfs:/path/tenure_cd5_2006.csv",recurse=True)
# dbutils.fs.rm("dbfs:/path/tenure_cd_norm_0.csv",recurse=True)
# dbutils.fs.rm("dbfs:/path/tenure_idr_0610.csv",recurse=True)
# dbutils.fs.rm("dbfs:/path/tenure_jif_0610.csv",recurse=True)
# dbutils.fs.rm("dbfs:/path/tenure_novel_0614.csv",recurse=True)
# dbutils.fs.rm("dbfs:/path/person_0707.csv",recurse=True)
# dbutils.fs.rm("dbfs:/path/aa_discipline_3level.csv",recurse=True)

# get file list
# import OS module
import os

# Get the list of all files and directories
path = "/dbfs/path/hbcu"
dir_list = os.listdir(path)
print(dir_list)

['full_hbcu.csv', 'hbcu_au_eid.csv', 'hbcu_au_eid_match.csv', 'hbcu_cd_pct.csv', 'hbcu_cd_pct_match.csv', 'hbcu_citepct.csv', 'hbcu_citepct_match.csv', 'hbcu_collab_af.csv', 'hbcu_collab_af_match.csv', 'hbcu_collab_afnum.csv', 'hbcu_collab_fresh.csv', 'hbcu_collab_senior.csv', 'hbcu_doi.csv', 'hbcu_eid.csv', 'hbcu_eid_match.csv', 'hbcu_match_doi.csv', 'hbcu_match_name.csv', 'hbcu_multiinst_info.csv', 'hbcu_novel.csv', 'hbcu_novel_match.csv', 'hbcu_person_name.csv', 'multi_inst_disambiguate.csv']


In [ ]:
df_au_eid = pd.read_csv(
    r"/dbfs/path/hbcu/hbcu_au_eid.csv"
)

df_eid = pd.read_csv(
    r"/dbfs/path/hbcu/hbcu_eid.csv"
)

df_eid_citepct = pd.read_csv(
    r"/dbfs/path/hbcu/hbcu_citepct.csv"
)

df_sm_map =pd.read_csv(
    r"/dbfs/path/sm_discipline_lower.csv")

df_jif = pd.read_csv(
    r"/dbfs/path/tenure_jif_0610.csv"
)

df_novel = pd.read_csv(
    r"/dbfs/path/hbcu/hbcu_novel.csv"
)

df_cd = pd.read_csv(
    r"/dbfs/path/hbcu/hbcu_cd_pct.csv"
)

df_multiinst = pd.read_csv(r"/dbfs/path/hbcu/hbcu_multiinst_info.csv")
df_person = pd.read_csv(r"/dbfs/path/hbcu/full_hbcu.csv")


In [ ]:
df_au_eid_match = pd.read_csv(
    r"/dbfs/path/hbcu/hbcu_au_eid_match.csv"
)

df_eid_match = pd.read_csv(
    r"/dbfs/path/hbcu/hbcu_eid_match.csv"
)

df_eid_citepct_match = pd.read_csv(
    r"/dbfs/path/hbcu/hbcu_citepct_match.csv"
)

df_jif = pd.read_csv(
    r"/dbfs/path/tenure_jif_0610.csv"
)

df_novel_match = pd.read_csv(
    r"/dbfs/path/hbcu/hbcu_novel_match.csv"
)

df_cd_match = pd.read_csv(
    r"/dbfs/path/hbcu/hbcu_cd_pct_match.csv"
)

In [ ]:
# mark pubs in 1 and 2 stages

t = df_au_eid.query('year >= 2010 and year <= 2020').merge(df_multiinst, on=['PersonId'], how='inner')

def get_stage(row):
  vars = [row['year'], row['year_start1'], row['year_end1'], row['year_start2'], row['year_end2']]
  for i, var in enumerate(vars):
    if isinstance(var, str) or var==np.nan:
      vars[i] = 0

  year, year_start1, year_end1, year_start2, year_end2 = vars[0], vars[1], vars[2], vars[3], vars[4]
  # 0: before move
  if year <= year_end1 and year >= year_start1:
    return 0
  # 1: after move
  if year <= year_end2 and year >= year_start2:
    return 1
  else:
    return np.nan
  
t['stage'] = t.apply(lambda row: get_stage(row), axis=1)
t


,Unnamed: 0_x,eid,PersonId,auid,year,Authorseq,last_author_seq,publication_type,doi,isAApub,Unnamed: 0_y,univ1,year_start1,year_end1,isHBCU1,univ2,year_start2,year_end2,isHBCU2,MoveType,period1_len,period2_len,stage
0,1,84857554607,886826,38860994900,2011,1,2,cp,10.1109/pimrc.2011.6139871,1.0,1563,North Carolina Agricultural and Technical Stat...,2018,2020,1,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0
1,2,85083075356,886826,38860994900,2020,1,1,cp,10.1109/ccwc47524.2020.9031127,1.0,1563,North Carolina Agricultural and Technical Stat...,2018,2020,1,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0
2,4,85035106301,886826,38860994900,2018,2,6,cp,10.1109/comst.2017.2773628,1.0,1563,North Carolina Agricultural and Technical Stat...,2018,2020,1,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0
3,7,84883418788,886826,38860994900,2013,1,2,ar,10.1109/tsp.2013.2272554,1.0,1563,North Carolina Agricultural and Technical Stat...,2018,2020,1,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0
4,10,85028758952,886826,38860994900,2017,4,4,ar,10.1109/tvt.2016.2619418,1.0,1563,North Carolina Agricultural and Technical Stat...,2018,2020,1,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27915,38147,80755185318,244503,7402187840,2011,1,2,ar,NaN,NaN,1475,North Carolina Agricultural and Technical Stat...,2011,2013,1,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0
27916,38401,84862003184,755214,37088014700,2012,1,1,ar,NaN,NaN,257,University of Maryland Eastern Shore,2014,2020,1,NaN,NaN,NaN,NaN,S,7.0,NaN,0.0
27917,38744,84868706388,802399,14069879700,2012,1,3,ar,NaN,NaN,3077,Jackson State University,2017,2020,1,NaN,NaN,NaN,NaN,S,4.0,NaN,0.0
27918,38861,78650232471,750841,6602222874,2010,5,5,ar,NaN,NaN,1827,Florida A&M University,2013,2020,1,NaN,NaN,NaN,NaN,S,8.0,NaN,0.0


In [ ]:
t.query("MoveType != 'S'")[['PersonId', 'univ1', 'MoveType']].drop_duplicates().sort_values('PersonId').set_index('PersonId').to_dict()

{'univ1': {10889: 'Tennessee State University',
  23054: 'Florida State University',
  24853: 'University of Alabama, The',
  26047: 'Howard University',
  29108: 'Howard University',
  29225: 'Florida State University',
  32171: 'Catholic University of America, The',
  48732: 'Hampton University',
  49322: 'Washington University in St. Louis',
  67546: 'Rensselaer Polytechnic Institute',
  76788: 'Princeton Theological Seminary',
  89564: 'Texas Southern University',
  100496: 'Jackson State University',
  115722: 'Michigan State University',
  118148: 'Florida State University',
  125935: 'Hampton University',
  148854: 'Howard University',
  150258: 'Mississippi State University',
  167279: 'Morehouse School of Medicine',
  172002: 'Beijing Institute of Technology',
  207451: 'University of Central Florida',
  212017: 'University of Idaho',
  220276: 'Western Michigan University',
  238431: 'Florida A&M University',
  238894: 'Florida International University',
  239416: 'Florida St

In [ ]:
# mark stages for control group
df_match = pd.read_csv(
    r"/dbfs/path/hbcu/hbcu_match_name.csv"
)

# drop duplicates
#df_match = df_match.sample(frac=1, random_state=1).drop_duplicates(subset=['PersonId'])

t_match = df_au_eid_match.query('year >= 2010 and year <= 2020').merge(df_match[['PersonId', 'PersonId_control']], on='PersonId_control').merge(t[['PersonId', 'year_start1', 'year_end1', 'year_start2', 'year_end2', 'MoveType']], on='PersonId').drop_duplicates()

t_match['stage'] = t_match.apply(lambda row: get_stage(row), axis=1)
t_match


,Unnamed: 0,eid,PersonId_control,auid,year,Authorseq,last_author_seq,publication_type,doi,isAApub,PersonId,year_start1,year_end1,year_start2,year_end2,MoveType,stage
0,0,85080024884,30039,9279801600,2020,3,5,ar,10.1016/j.bushor.2020.01.009,1.0,24853,1999,2014,2015.0,2020.0,M-PWI-HBCU,1.0
11,2,84900854898,30039,9279801600,2014,3,3,ar,10.1108/jwl-08-2013-0057,1.0,24853,1999,2014,2015.0,2020.0,M-PWI-HBCU,0.0
22,4,84925100394,30039,7006724172,2015,2,2,ar,10.1080/0965254x.2014.914067,1.0,24853,1999,2014,2015.0,2020.0,M-PWI-HBCU,1.0
33,5,82555197073,30039,7006724172,2011,2,4,ar,10.1108/08858621211188948,1.0,24853,1999,2014,2015.0,2020.0,M-PWI-HBCU,0.0
44,8,79957896997,30039,7006724172,2011,4,4,ar,10.1108/10610421111134923,1.0,24853,1999,2014,2015.0,2020.0,M-PWI-HBCU,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468798,11878,85044068680,627531,8374772200,2014,1,3,ar,10.1080/10437797.2014.947899,1.0,620868,2012,2018,2018.0,2020.0,M-HBCU-PWI,0.0
468820,11880,77951229585,627531,8374772200,2010,3,5,ar,10.1080/00377310903504940,1.0,620868,2012,2018,2018.0,2020.0,M-HBCU-PWI,0.0
468842,11887,84955258941,627531,8374772200,2016,2,3,ar,10.1080/10437797.2016.1119016,1.0,620868,2012,2018,2018.0,2020.0,M-HBCU-PWI,0.0
468864,11905,84929879065,627531,8374772200,2015,1,3,ar,10.1080/15228835.2015.1022683,1.0,620868,2012,2018,2018.0,2020.0,M-HBCU-PWI,0.0


In [ ]:
# concat tables
t_match['is_control'] = 1
t_match = t_match.drop(columns=['PersonId']).rename(columns={'PersonId_control': 'PersonId'})
t = pd.concat([t, t_match]).fillna({'is_control':0}).drop_duplicates()

t

,Unnamed: 0_x,eid,PersonId,auid,year,Authorseq,last_author_seq,publication_type,doi,isAApub,Unnamed: 0_y,univ1,year_start1,year_end1,isHBCU1,univ2,year_start2,year_end2,isHBCU2,MoveType,period1_len,period2_len,stage,Unnamed: 0,is_control
0,1.0,84857554607,886826,38860994900,2011,1,2,cp,10.1109/pimrc.2011.6139871,1.0,1563.0,North Carolina Agricultural and Technical Stat...,2018,2020,1.0,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0,NaN,0.0
1,2.0,85083075356,886826,38860994900,2020,1,1,cp,10.1109/ccwc47524.2020.9031127,1.0,1563.0,North Carolina Agricultural and Technical Stat...,2018,2020,1.0,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0,NaN,0.0
2,4.0,85035106301,886826,38860994900,2018,2,6,cp,10.1109/comst.2017.2773628,1.0,1563.0,North Carolina Agricultural and Technical Stat...,2018,2020,1.0,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0,NaN,0.0
3,7.0,84883418788,886826,38860994900,2013,1,2,ar,10.1109/tsp.2013.2272554,1.0,1563.0,North Carolina Agricultural and Technical Stat...,2018,2020,1.0,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0,NaN,0.0
4,10.0,85028758952,886826,38860994900,2017,4,4,ar,10.1109/tvt.2016.2619418,1.0,1563.0,North Carolina Agricultural and Technical Stat...,2018,2020,1.0,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468798,NaN,85044068680,627531,8374772200,2014,1,3,ar,10.1080/10437797.2014.947899,1.0,NaN,NaN,2012,2018,NaN,NaN,2018.0,2020.0,NaN,M-HBCU-PWI,NaN,NaN,0.0,11878.0,1.0
468820,NaN,77951229585,627531,8374772200,2010,3,5,ar,10.1080/00377310903504940,1.0,NaN,NaN,2012,2018,NaN,NaN,2018.0,2020.0,NaN,M-HBCU-PWI,NaN,NaN,0.0,11880.0,1.0
468842,NaN,84955258941,627531,8374772200,2016,2,3,ar,10.1080/10437797.2016.1119016,1.0,NaN,NaN,2012,2018,NaN,NaN,2018.0,2020.0,NaN,M-HBCU-PWI,NaN,NaN,0.0,11887.0,1.0
468864,NaN,84929879065,627531,8374772200,2015,1,3,ar,10.1080/15228835.2015.1022683,1.0,NaN,NaN,2012,2018,NaN,NaN,2018.0,2020.0,NaN,M-HBCU-PWI,NaN,NaN,0.0,11905.0,1.0


In [ ]:
tables1 = [df_au_eid,df_eid,
df_eid_citepct,
df_novel,
df_cd]

tables2 = [df_au_eid_match,df_eid_match,
df_eid_citepct_match,
df_novel_match,
df_cd_match]

for i in range(len(tables1)):
  table1 = tables1[i]
  table2 = tables2[i]
  table2 = table2.rename(columns={'PersonId_control': 'PersonId'})
  table1 = pd.concat([table1, table2]).drop_duplicates()
  tables1[i] = table1

df_au_eid, df_eid,df_eid_citepct,df_novel,df_cd = tables1[0], tables1[1], tables1[2], tables1[3], tables1[4]

# utilities

In [ ]:
# confidence interval
def calculate_confidence_interval(group):
    std = group.std()
    n = len(group)
    conf_level = 0.95
    dof = n - 1
    t_value = tstats.ppf((1 + conf_level) / 2, dof)
    se = std / np.sqrt(n)
    margin_error = t_value * se
    return margin_error


def winsorize_group(group):
    return winsorize(group, limits=[0, 0.05])


In [ ]:
import plotly.colors as pcolors

def generate_color_list(num_colors):
    # Get a list of colors from the Plotly color scales
    color_scale = pcolors.qualitative.Set1
    colors = color_scale * (num_colors // len(color_scale) + 1)

    color_list = []
    # Modify the colors to include the desired opacity
    for i in range(num_colors):
        color_list.append(f"rgba{colors[i][3:-1]}" + ", {})")
    return color_list

def generate_symbol_list(num_symbols):
    # Define a set of symbols to use
    symbols = ['circle', 'square', 'diamond', 'cross', 'x', 'star', 'triangle-up', 'triangle-down']

    # Repeat the symbols to match the desired number
    symbol_list = symbols * (num_symbols // len(symbols) + 1)
    return symbol_list[:num_symbols]

# Example usage
num_items = 9
colors = generate_color_list(num_items)
symbols = generate_symbol_list(num_items)

# Print the generated lists
print(colors)
print(symbols)


['rgba(228,26,28, {})', 'rgba(55,126,184, {})', 'rgba(77,175,74, {})', 'rgba(152,78,163, {})', 'rgba(255,127,0, {})', 'rgba(255,255,51, {})', 'rgba(166,86,40, {})', 'rgba(247,129,191, {})', 'rgba(153,153,153, {})']
['circle', 'square', 'diamond', 'cross', 'x', 'star', 'triangle-up', 'triangle-down', 'circle']


In [ ]:
def plot_styling(
    fig,
    size=(250, 270),
    title=None,
    xtitle="Year relative to move",
    dtick=None,
):
    layout_params = dict(
      title=dict(text=title, x=0.5, xanchor='center', xref='container', font=dict(size=14)),
        width=size[1],
        height=size[0],
        template='simple_white',
        font=dict(size=11, family="Arial"),
        margin=dict(l=50, r=50, b=50, t=50),
    )

    fig.update_layout(
        **layout_params
    )

    fig.update_xaxes(title=dict(text=xtitle, standoff=0), dtick=dtick, minor=dict(ticklen=3, dtick=1))

    return fig

In [ ]:
def add_lines_with_errorband(fig, x, y, lower, upper, name, color, text=None, colors=colors, showlegend=False, showband=True, dash=None, **kwargs):
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode="lines+text",
            text=text,
            textposition='top left',
            name=name,
            line=dict(color=colors[color].format(1), width=1, dash=dash),
            showlegend=showlegend,
        ),**kwargs,
    )

    # add error band
    if showband:
        x_error = np.concatenate((x, x[::-1]))
        y_error = np.concatenate((lower, upper[::-1]))
        fig.add_trace(
            go.Scatter(
                x=x_error,
                y=y_error,
                fill="toself",
                fillcolor=colors[color].format(0.2),
                line=dict(color="rgba(255,255,255,0)"),
                showlegend=False,
            ),**kwargs,
        )

    return fig

# probability analysis

In [ ]:
# based table

df_base = (
    t[
        [
            "PersonId",
            "year_end1",
            "MoveType",
            'is_control',
            'stage',
        ]
    ]
    .rename(columns={"year_end1": 'year_asst'})
    .merge(
        df_au_eid[["PersonId", "eid", "year", "Authorseq", "last_author_seq"]],
        on=["PersonId"],
    )
    .merge(
        df_eid[
            [
                "PersonId",
                "eid",
                "srcid",
                "subfield_hybrid",
                "n_references",
                "avgnum_past_eid",
                "publication_type",
            ]
        ],
        on=["PersonId", "eid"],
    )
        .assign(tau=lambda t: t["year"] - t["year_asst"])
        # only key author
        #.query("Authorseq == 1 or Authorseq == last_author_seq")
    .drop_duplicates()
)
df_base['all'] = 1

tau_range = np.arange(-4, 5, 1)

In [ ]:
# statistical analysis

a = df_base.query("MoveType == 'M-HBCU-PWI' and is_control == 0")['PersonId'].nunique()
b = df_base.query("MoveType == 'M-PWI-HBCU' and is_control == 0")['PersonId'].nunique()
c = df_base.query("(MoveType == 'M-HBCU-PWI' or MoveType == 'M-PWI-HBCU') and is_control == 0")['eid'].nunique()
d = df_base.query("(MoveType == 'M-HBCU-PWI' or MoveType == 'M-PWI-HBCU') and is_control == 1")['eid'].nunique()

a, b, c, d

(49, 59, 4269, 12194)

## productivity

In [ ]:
def fill_nan_with_zero(val, ind, col):
    if pd.isna(val):
        person_id = ind
        tau_range = t_range[t_range['PersonId'] == person_id]
        if not tau_range.empty:
            min_tau = tau_range['min_tau'].values[0]
            max_tau = tau_range['max_tau'].values[0]
            if min_tau <= col <= max_tau:
                return 0
    return val

In [ ]:
groupvar, group = 'is_control', [0, 1]
df_norms = []

for is_control in [0, 1]:
    for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
        t = df_base.query("MoveType	== @movetype and is_control == @is_control")[[groupvar, "PersonId", "tau", "year_asst", "eid"]].drop_duplicates()

        # get min and max tau
        t['min_tau'] = t['year_asst'].apply(lambda x: max(tau_range[0], 2006-x))
        t['max_tau'] = t['year_asst'].apply(lambda x: min(tau_range[-1], 2020-x))
        t_range = t[['PersonId', 'min_tau', 'max_tau']].drop_duplicates()

        df_norm = (
            t.groupby([groupvar, "PersonId", "tau"])
            .agg(mean1=("eid", "nunique"))
            .reset_index()
            .query("tau in @tau_range")
            .pivot(columns=['tau'], values=['mean1'], index=[groupvar, "PersonId"])
        )
        # fill 0 for tau range
        for ind, row in df_norm.iterrows():
            for col in df_norm.columns:
                val = row[col]
                row[col] = fill_nan_with_zero(val, ind[-1], col[-1])

        df_norm = (df_norm
            .stack()
            .reset_index()
            .dropna()
            .groupby([groupvar, "tau"])
            .agg(
                mean2=("mean1", "mean"),
                confint=("mean1", calculate_confidence_interval),
            )
            .reset_index()
        )

        df_norm['is_control'] = is_control
        df_norm['movetype'] = movetype

        df_norms.append(df_norm)

t = pd.concat(df_norms)
t

,is_control,tau,mean2,confint,movetype
0,0,-4,2.816327,1.241923,M-HBCU-PWI
1,0,-3,3.408163,1.570967,M-HBCU-PWI
2,0,-2,3.020408,1.531714,M-HBCU-PWI
3,0,-1,3.265306,1.377244,M-HBCU-PWI
4,0,0,3.244898,1.152714,M-HBCU-PWI
5,0,1,3.125000,1.228471,M-HBCU-PWI
6,0,2,3.086957,1.402453,M-HBCU-PWI
7,0,3,4.139535,1.746847,M-HBCU-PWI
8,0,4,3.897436,1.410631,M-HBCU-PWI
0,0,-4,2.661017,1.313399,M-PWI-HBCU


In [ ]:
# plot
y_range = (0, np.ceil((t['mean2'] + t['confint']).max()))

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
    df_norm = t.query("movetype == @movetype")
    fig = go.Figure()
    for i, domain in enumerate(group):
        t1_domain = df_norm.query(f"{groupvar} == @domain")
        x = tau_range[:]
        y1 = t1_domain["mean2"].to_numpy()
        y1_upper, y1_lower = (
            t1_domain["mean2"].to_numpy() + t1_domain["confint"].to_numpy(),
            t1_domain["mean2"].to_numpy() - t1_domain["confint"].to_numpy(),
        )

        # get color
        if domain == 1:
            color = 8
        elif movetype == 'M-HBCU-PWI':
            color = 1
        else:
            color = 4

        fig = add_lines_with_errorband(
            fig, x, y1, upper=y1_upper, lower=y1_lower, name=domain, color=color, showlegend=False
        )

    fig = plot_styling(fig, size=(250, 250), title=None)

    fig.update_xaxes(range=(tau_range[0], tau_range[-1]), mirror=True, zeroline=True, zerolinecolor="rgba(128,128,128, 0.3)")
    fig.update_yaxes(
        title=dict(text='Annual publications', standoff=0),
        titlefont=dict(color="black"),
        tickfont=dict(color="black"),
        range=y_range,
        mirror=True,
        zeroline=True,
        zerolinecolor="rgba(128,128,128, 0.3)",
    )

    fig.show()

## hit prob
top 10% - highly cited papers

In [ ]:
# get their tau and cite

groupvar, group = 'is_control', [0, 1]
cite_metric, jif_metric, novel_metric, cd_metric = "cite3_pct", 'jif_pct', "novelty_pct", 'cd5_pct'
metrics = [cite_metric, jif_metric, novel_metric, cd_metric]

t_cite = (
    df_base.merge(
        df_eid_citepct[["eid", "year", "subfield_hybrid", cite_metric]],
        on=["eid", "year", "subfield_hybrid"],
    )
    #.merge(df_person_diffbin[['PersonId', 'label']], on='PersonId')
    .drop_duplicates()
)

t_jif = (
    df_base
    .merge(df_jif[['srcid', 'year', jif_metric]], on=['srcid', "year"])
    # only use journal article
    #.query("sourcetype == 'j'")
    .drop_duplicates()
)

t_novel = (
    df_base.merge(
        df_novel[["eid", novel_metric]],
        on=["eid"],
        how='inner'
    )
    .drop_duplicates()
)

t_cd = (
    df_base
    # remove 2006
    .query("year > 2006")
    .merge(df_cd[['eid', cd_metric]], on=["eid"])
    # only use journal article
    #.query("sourcetype == 'j'")
    .drop_duplicates()
)

ts = [t_cite, t_jif, t_novel, t_cd]
for i, t in enumerate(ts):
  metric = metrics[i]
  t['all'] = 1
  ts[i] = t

In [ ]:
# get their tau and cite
df_norms = []
thresholds = {"cite3_pct": 0.9, 'jif_pct': 0.9, "novelty_pct": 0.9, 'cd5_pct': 0.9}
for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
    for method in ['all']: #, 'max', 'min'
        for i, metric in enumerate(metrics):
            t = ts[i].query("MoveType == @movetype")
            # top quartile
            if method == 'max':
                t = t[t[metric] >= t.groupby([groupvar, "PersonId", "tau"])[metric].transform('max')]
            # bottom quartile
            if method == 'min':
                t = t[t[metric] <= t.groupby([groupvar, "PersonId", "tau"])[metric].transform('min')]
            t = t[[groupvar, 'PersonId', "tau", 'eid', metric]].drop_duplicates()
            df_norm = (
                t
                .query("tau in @tau_range")
                .groupby([groupvar, "tau"])
                .agg(
                    mean2=(metric, lambda x: sum(x >= thresholds[metric]) / len(x)),
                )
                .reset_index()
            )
            df_norm['movetype'] = movetype
            df_norm['method'] = method
            df_norm['metric'] = metric
            df_norms.append(df_norm)

df_norms = pd.concat(df_norms)

In [ ]:
# plot
ylabels = [
    "% top-citation pubs",
    "% top-IF pubs",
    "% high-novelty pubs",
    "% high-CD5 pubs",
]

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1)
    for j, metric in enumerate(metrics):
        for k, method in enumerate(["all"]): #, "max", "min"
            for i, domain in enumerate(group):
                t1_domain = df_norms.query(
                    f"{groupvar} == @domain and movetype == @movetype and metric == @metric and method == @method"
                )
                x = tau_range[:]
                y1 = t1_domain["mean2"].to_numpy()

                # get color
                if domain == 1:
                    color = 8
                elif movetype == 'M-HBCU-PWI':
                    color = 1
                else:
                    color = 4

                fig = add_lines_with_errorband(
                    fig,
                    x,
                    y1,
                    upper=None,
                    lower=None,
                    name=domain,
                    color=color,
                    showlegend=False,
                    showband=False,
                    row=k + 1,
                    col=j + 1,
                )

        fig.update_yaxes(
            title=dict(text=ylabels[j], standoff=0),
            col=j + 1,
        )


    fig = plot_styling(fig, size=(200, 800), title=None)
    fig.update_xaxes(mirror=True, zeroline=True, zerolinecolor="rgba(128,128,128, 0.3)")
    fig.update_yaxes(
        titlefont=dict(color="black"),
        tickfont=dict(color="black"),
        mirror=True,
        zeroline=True,
        zerolinecolor="rgba(128,128,128, 0.3)",
    )

    # use % y axis
    fig.update_yaxes(tickformat=",.0%")

    fig.show()

In [ ]:
df_norms = []
metrics = [cite_metric, jif_metric, novel_metric, cd_metric]

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
    for method in ['mean']: #, 'max', 'min'
        for i, metric in enumerate(metrics):
            df_norm = (
                ts[i].query("MoveType == @movetype")
                .groupby([groupvar, "PersonId", "tau"])
                .agg(mean1=(metric, method))
                .reset_index()
                .query("tau in @tau_range")
                .groupby([groupvar, "tau"])
                .agg(
                    mean2=("mean1", "mean"),
                    confint=("mean1", calculate_confidence_interval),
                )
                .reset_index()
            )
            df_norm['movetype'] = movetype
            df_norm['method'] = method
            df_norm['metric'] = metric
            df_norms.append(df_norm)

df_norms = pd.concat(df_norms)

In [ ]:
# plot
ylabels = ["Citation pctl.", "JIF pctl.", "Novelty pctl.", "CD5 pctl."]

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1)

    for j, metric in enumerate(metrics):
        for k, method in enumerate(["mean"]): #, "max", "min"
            for i, domain in enumerate(group):

                t1_domain = df_norms.query(
                    f"{groupvar} == @domain and movetype == @movetype and metric == @metric and method == @method"
                )
                x = tau_range[:]
                y1 = t1_domain["mean2"].to_numpy()
                y1_upper, y1_lower = (
                    t1_domain["mean2"].to_numpy() + t1_domain["confint"].to_numpy(),
                    t1_domain["mean2"].to_numpy() - t1_domain["confint"].to_numpy(),
                )

                fig = add_lines_with_errorband(
                    fig,
                    x,
                    y1,
                    upper=y1_upper,
                    lower=y1_lower,
                    name=domain,
                    color=i,
                    showlegend=False,
                    row=k + 1,
                    col=j + 1,
                )
            fig.update_yaxes(
                title=dict(text=ylabels[j], standoff=0),
                col=j + 1,
            )


    fig = plot_styling(fig, size=(200, 800), title=None)
    fig.update_xaxes(range=(tau_range[0], tau_range[-1]), mirror=True, zeroline=True, zerolinecolor="rgba(128,128,128, 0.3)")
    fig.update_yaxes(
        titlefont=dict(color="black"),
        tickfont=dict(color="black"),
        mirror=True,
        zeroline=True,
        zerolinecolor="rgba(128,128,128, 0.3)",
    )

    fig.show()

## collab

In [ ]:
# HBCU-PWI collaboration

In [ ]:
# mark if papers are HBCU-PWI collaboration
df_collab_af = pd.read_csv(
    f"/dbfs/path/hbcu/hbcu_collab_af.csv"
)
# only keep US univ
df_collab_af = df_collab_af.query("country=='United States'")
# df_collab_af.groupby('afname').agg(n=("eid", 'nunique')).sort_values("n", ascending=False).head(20)

HBCU_list = [
    "Howard University",
    "Florida State University",
    "Florida Agricultural and Mechanical University",
    "Jackson State University",
    "North Carolina Agricultural and Technical State University",
    "North Carolina A and T State University",
    "FAMU-FSU College of Engineering",
    "Morgan State University",
    "Hampton University",
    "University of Maryland Eastern Shore",
    "Texas Southern University",
    "Tennessee State University",
]

# if univ name contains hbcu, set 1
def get_if_hbcu(s):
    if not isinstance(s, str):
        return np.nan
    for u in HBCU_list:
        if u in s:
            return 1
    return 0


df_collab_af["is_hbcu"] = df_collab_af["afname"].apply(get_if_hbcu)
df_collab_af = df_collab_af.dropna(subset=["is_hbcu"])

# aggregate: if all 0, "PWI"; if all 1, 'HBCU', if 0 and 1, 'collab'
def classify_institution(group):
    if group['is_hbcu'].all() == 1:
        return 'HBCU'
    elif group['is_hbcu'].any() == 0:
        return 'PWI'
    else:
        return 'collab'

# Group by 'eid' and apply the custom function
result = df_collab_af.groupby('eid').apply(classify_institution).reset_index(name='institution_type')
result['institution_type'].value_counts()

collab    8914
HBCU      8098
PWI       7068
Name: institution_type, dtype: int64

In [ ]:
# check distribution
t = df_base[['PersonId', 'MoveType', 'eid', 'tau', 'stage']].merge(result, on=['eid'])

# groupby to check percentage
mt = ['M-HBCU-PWI', 'M-PWI-HBCU']
t_pct = t.query("MoveType in @ mt and tau in @tau_range and stage in [0, 1]").groupby(['MoveType', 'tau', 'institution_type']).agg(n=('eid', 'nunique')).reset_index()
t_sum = t.query("MoveType in @ mt and tau in @tau_range and stage in [0, 1]").groupby(['MoveType', 'tau']).agg(n=('eid', 'nunique')).reset_index()
t_pct = t_pct.merge(t_sum, on=['MoveType', 'tau'])
t_pct['pct'] = t_pct['n_x'] / t_pct['n_y']
t_pct.query("institution_type == 'collab'")

,MoveType,tau,institution_type,n_x,n_y,pct
2,M-HBCU-PWI,-4,collab,54,128,0.421875
5,M-HBCU-PWI,-3,collab,66,172,0.383721
8,M-HBCU-PWI,-2,collab,65,149,0.436242
11,M-HBCU-PWI,-1,collab,79,172,0.459302
14,M-HBCU-PWI,0,collab,86,163,0.527607
17,M-HBCU-PWI,1,collab,70,140,0.500000
20,M-HBCU-PWI,2,collab,40,135,0.296296
23,M-HBCU-PWI,3,collab,30,159,0.188679
26,M-HBCU-PWI,4,collab,32,149,0.214765
28,M-PWI-HBCU,-4,collab,15,136,0.110294


In [ ]:
# control group
# mark if papers are HBCU-PWI collaboration
df_collab_af = pd.read_csv(
    f"/dbfs/path/hbcu/hbcu_collab_af_match.csv"
)
# only keep US univ
df_collab_af = df_collab_af.query("country=='United States'")
# df_collab_af.groupby('afname').agg(n=("eid", 'nunique')).sort_values("n", ascending=False).head(20)

df_collab_af["is_hbcu"] = df_collab_af["afname"].apply(get_if_hbcu)
df_collab_af = df_collab_af.dropna(subset=["is_hbcu"])

# Group by 'eid' and apply the custom function
result = df_collab_af.groupby('eid').apply(classify_institution).reset_index(name='institution_type')

# check distribution
t = df_base[['PersonId', 'MoveType', 'eid', 'tau', 'stage']].merge(result, on=['eid'])

# groupby to check percentage
mt = ['M-HBCU-PWI', 'M-PWI-HBCU']
t_pct2 = t.query("MoveType in @ mt and tau in @tau_range and stage in [0, 1]").groupby(['MoveType', 'tau', 'institution_type']).agg(n=('eid', 'nunique')).reset_index()
t_sum = t.query("MoveType in @ mt and tau in @tau_range and stage in [0, 1]").groupby(['MoveType', 'tau']).agg(n=('eid', 'nunique')).reset_index()
t_pct2 = t_pct2.merge(t_sum, on=['MoveType', 'tau'])
t_pct2['pct'] = t_pct2['n_x'] / t_pct2['n_y']

# final
t_pct = t_pct.query("institution_type == 'collab'")
t_pct2 = t_pct2.query("institution_type == 'collab'")
t_pct['group'] = 'treat'
t_pct2['group'] = 'control'

t_pct = pd.concat([t_pct, t_pct2])
t_pct

,MoveType,tau,institution_type,n_x,n_y,pct,group
2,M-HBCU-PWI,-4,collab,54,128,0.421875,treat
5,M-HBCU-PWI,-3,collab,66,172,0.383721,treat
8,M-HBCU-PWI,-2,collab,65,149,0.436242,treat
11,M-HBCU-PWI,-1,collab,79,172,0.459302,treat
14,M-HBCU-PWI,0,collab,86,163,0.527607,treat
17,M-HBCU-PWI,1,collab,70,140,0.500000,treat
20,M-HBCU-PWI,2,collab,40,135,0.296296,treat
23,M-HBCU-PWI,3,collab,30,159,0.188679,treat
26,M-HBCU-PWI,4,collab,32,149,0.214765,treat
28,M-PWI-HBCU,-4,collab,15,136,0.110294,treat


In [ ]:
# plot
groupvar, group = 'group', ['treat', 'control']
t = t_pct
y_range = (0, 0.65)

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
    fig = go.Figure()
    for domain in group:
        t1_domain = t.query(
            f"MoveType == @movetype and {groupvar} == @domain"
        )
        x = t1_domain['tau'].to_numpy()
        y = t1_domain["pct"].to_numpy()

        # get color
        if domain == 'control':
            color = 8
        elif movetype == 'M-HBCU-PWI':
            color = 1
        else:
            color = 4

        fig=add_lines_with_errorband(fig, x, y, lower=None, upper=None, name=None, color=color, text=None, colors=colors, showlegend=False, showband=False)

    fig = plot_styling(fig, size=(200, 220), title=None)
    fig.update_layout(legend=dict(orientation="h", x=0.5, xanchor='center', y=-0.2, yanchor='top'))
    fig.update_xaxes(mirror=True, zeroline=True, zerolinecolor="rgba(128,128,128, 0.3)")
    fig.update_yaxes(
        title=dict(text='HBCU & PWI<br>collaboration', standoff=0),
        titlefont=dict(color="black"),
        tickfont=dict(color="black"),
        tickformat=",.0%",
        mirror=True,
        zeroline=True,
        zerolinecolor="rgba(128,128,128, 0.3)",
        range=y_range,
    )

    fig.show()

## team size

In [ ]:
# team size
t_size = (
    df_base.query("MoveType in @ mt and tau in @tau_range and stage in [0, 1]")[["MoveType", "PersonId", "tau", "eid", "last_author_seq"]]
    # clip tau
    .query("tau in @tau_range").drop_duplicates()
)

def get_team_type(x):
  if x == 1:
    return 'Single'
  if x > 1:
    return 'Collab'
  return np.nan

t_size['team_type'] = t_size['last_author_seq'].apply(get_team_type)
t_size

,MoveType,PersonId,tau,eid,last_author_seq,team_type
2302,M-HBCU-PWI,620863,4,85068004540,1,Single
2318,M-HBCU-PWI,620863,0,84925202221,1,Single
16573,M-HBCU-PWI,754286,-1,85006493508,3,Collab
16713,M-HBCU-PWI,754286,-3,85006493508,3,Collab
16993,M-HBCU-PWI,754286,-2,84992316033,3,Collab
...,...,...,...,...,...,...
6688560,M-PWI-HBCU,106123,-4,60949473794,1,Single
6688566,M-PWI-HBCU,106123,-4,44049087595,1,Single
6688579,M-PWI-HBCU,124865,3,84942033014,1,Single
6688587,M-PWI-HBCU,124865,-4,40949106101,1,Single


In [ ]:
t_pct = t_size.groupby(['MoveType', 'tau', 'team_type']).agg(n=('eid', 'nunique')).reset_index()
t_sum = t_size.groupby(['MoveType', 'tau']).agg(n=('eid', 'nunique')).reset_index()
t_pct = t_pct.merge(t_sum, on=['MoveType', 'tau'])
t_pct['pct'] = t_pct['n_x'] / t_pct['n_y']

# plot
t = t_pct

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:  
  fig = go.Figure()
  for team_type in ['Single', 'Collab']:
    t1_domain = t.query(
          "MoveType == @movetype and team_type == @team_type"
      )
    x = t1_domain['tau'].to_numpy()
    y = t1_domain["pct"].to_numpy()

    fig.add_trace(go.Scatter(x=x, y=y, name=team_type))

  fig = plot_styling(fig, size=(250, 300), title=None)
  fig.update_xaxes(mirror=True, zeroline=True, zerolinecolor="rgba(128,128,128, 0.3)")
  fig.update_yaxes(
      titlefont=dict(color="black"),
      tickfont=dict(color="black"),
      mirror=True,
      zeroline=True,
      zerolinecolor="rgba(128,128,128, 0.3)",
  )

  fig.show()

In [ ]:


# fresh collaborator
df_fresh_collab = pd.read_csv(
    f"/dbfs/path/hbcu/hbcu_collab_fresh.csv"
)
t_fresh = (
    df_base.merge(df_fresh_collab, on=["PersonId", "eid"])[
        ["MoveType", groupvar, "PersonId", "tau", "eid", "fresh_perc"]
    ].drop_duplicates()
    # clip tau
    .query("tau in @tau_range")
)

# % Cross af, Cross country papers

df_collab_af = pd.read_csv(
    f"/dbfs/path/hbcu/hbcu_collab_afnum.csv"
)
df_collab_af["isCross_af"] = df_collab_af["af_num"].apply(lambda x: 1 if x > 1 else 0)
df_collab_af["isCross_ctry"] = df_collab_af["ctry_num"].apply(
    lambda x: 1 if x > 1 else 0
)

t_af = (
    df_base.merge(df_collab_af, on=["eid"])[
        ["MoveType", groupvar, "PersonId", "tau", "eid", "isCross_af", "isCross_ctry"]
    ].drop_duplicates()
    # clip tau
    .query("tau in @tau_range")
)

# % junior

df_senior = pd.read_csv(
    f"/dbfs/path/hbcu/hbcu_collab_senior.csv"
)
df_senior["is_junior"] = df_senior["seniority"].apply(lambda x: 1 if x < 7 else 0)
numer = (
    df_senior.query("is_junior == 1")
    .groupby("eid")
    .agg(numer=("auid", "nunique"))
    .reset_index()
)
denom = df_senior.groupby("eid").agg(denom=("auid", "nunique")).reset_index()
df_senior = denom.merge(numer, on=["eid"], how="left").fillna(0)
df_senior["junior_pct"] = df_senior["numer"] / df_senior["denom"]

t_junior = (
    df_base.merge(df_senior, on=["eid"])[
        ["MoveType", groupvar, "PersonId", "tau", "eid", "junior_pct"]
    ].drop_duplicates()
    # clip tau
    .query("tau in @tau_range")
)

ts = [t_size, t_fresh, t_af, t_af, t_junior]
metrics = ["last_author_seq", "fresh_perc", "isCross_af", "isCross_ctry", "junior_pct"]

In [ ]:
# get every person every tau's mean/max arc
df_norms = []

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
  for i, t in enumerate(ts):
    metric = metrics[i]
    df_norm = (
        t.query("MoveType == @movetype").groupby([groupvar, "PersonId", "tau"])
        .agg(mean1=(metric, 'mean'))
        .reset_index()
        .query("tau in @tau_range")
        .groupby([groupvar, "tau"])
        .agg(
            mean2=("mean1", "mean"),
            confint=("mean1", calculate_confidence_interval),
        )
        .reset_index()
    )
    df_norm['movetype'] = movetype
    df_norm['metric'] = metric
    df_norms.append(df_norm)

df_norms = pd.concat(df_norms)

In [ ]:
# plot
ylabels = ["Avg. team size", 'Avg. % fresh collaborator', '% cross-affiliation', '% cross-country', "Avg. % junior"]
tickformats = [None, ",.0%", ",.0%", ",.0%", ",.0%"]

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.1)

    for j, metric in enumerate(metrics):
        for i, domain in enumerate(group):
            t1_domain = df_norms.query(
                f"{groupvar} == @domain and movetype == @movetype and metric == @metric"
            )
            x = tau_range[:]
            y1 = t1_domain["mean2"].to_numpy()

            fig = add_lines_with_errorband(
                fig,
                x,
                y1,
                upper=None,
                lower=None,
                name=domain,
                color=i,
                showlegend=False,
                showband=False,
                row=1,
                col=j + 1,
            )

        # mark 0-1 zone
        fig.add_vrect(
            x0=0,
            x1=1,
            line_width=0,
            fillcolor="gray",
            opacity=0.2,
            row=1,
            col=j + 1,
        )
        
        fig.update_yaxes(
            title=dict(text=ylabels[j], standoff=0),
            tickformat=tickformats[j],
            col=j + 1,
        )

    fig = plot_styling(fig, size=(230, 1100), title=None)
    fig.update_yaxes(
        titlefont=dict(color="black"),
        tickfont=dict(color="black"),
    )

    fig.show()